In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
CPUs = pd.read_csv("cpufinalv.csv")

In [23]:
CPUclean = CPUs.copy()
remove_spec = lambda a : a.strip()
create_list = lambda a : list(map(remove_spec,re.split('& |,|\\*|-', a)))
for col in ['Categories','SubCategories'] :
    CPUclean[col] = CPUclean[col].apply(create_list)


In [24]:
def cleaner(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
for col in ['Categories','SubCategories'] :
    CPUclean[col] = CPUclean[col].apply(cleaner)


In [25]:
def couple(x):
    return ' '.join(x['Categories']) + ' ' + ' '.join(x['SubCategories'])
CPUclean['Features'] = CPUclean.apply(couple, axis=1)
CPUclean.head(10)

,CpuName,Manufacturer,SingleScore,MultiScore,Cores,Threads,BaseClock,TurboClock,Type,Categories,Price,SubCategories,Features
0,Threadripper 3990X,AMD,1262,75671,64,128,2.9,4.50,Rendering moduling simulation streaming gaming...,[workstation],"4,861.00","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
1,Threadripper Pro 3995WX,AMD,1231,73220,64,128,2.7,4.20,Rendering moduling simulation streaming gaming...,[workstation],6866,"[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
2,Epyc 7702P,AMD,993,48959,64,128,2.0,3.35,Rendering moduling simulation streaming server,[workstation],"1,898","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
3,Threadripper 3970X,AMD,1308,46874,32,64,3.7,4.50,Rendering moduling simulation streaming gaming...,[workstation],"2,495.00","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
4,Threadripper Pro 3975WX,AMD,1244,43450,32,64,3.5,4.20,Rendering moduling simulation streaming gaming...,[workstation],"2,997","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
5,Threadripper 3960X,AMD,1307,34932,24,64,3.8,4.50,Rendering moduling simulation streaming gaming...,[workstation],"1,475","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
6,Threadripper 2990WX,AMD,1005,29651,32,64,3.0,4.20,Rendering moduling simulation streaming gaming...,[workstation],"2,000","[design, engineering, 3dmodeling, simulation, ...",workstation design engineering 3dmodeling simu...
7,Ryzen 9 5950X,AMD,1684,28782,16,32,3.4,4.90,Rendering moduling simulation streaming gaming,[workstation],441.71,[verycapableforallcategories],workstation verycapableforallcategories
8,Core i9 12900K,Intel,2007,27852,16,24,3.2,5.20,Rendering moduling simulation streaming gaming,[workstation],399.99,[verycapableforallcategories],workstation verycapableforallcategories
9,Core i9 12900KF,Intel,1974,27852,16,24,3.2,5.20,Rendering moduling simulation streaming gaming,[workstation],398.99,[verycapableforallcategories],workstation verycapableforallcategories


In [26]:
count_vector = CountVectorizer(stop_words="english")
matrix = count_vector.fit_transform(CPUclean['Features'])
Cosine_sim =  cosine_similarity(matrix,matrix)
#indeces = pd.Series(CPUs.index,index=CPUs["CpuName"]).drop_duplicates()
Cosine_sim_df = pd.DataFrame(Cosine_sim)

In [75]:
#simple Recommendation with sorting nor filtering 
def Recomm_System(title):
    a=CPUclean.copy()
    index = a[a['Features'] == title].index[0]
    print(index)
    TopIndeces = list(Cosine_sim_df[index].nlargest(10).index) 
    try:
        TopIndeces.remove(index)
    except:
        pass
    similar_df = a.iloc[TopIndeces][['CpuName']]
    similar_df['cosine_similarity']=Cosine_sim_df[index].iloc[TopIndeces]
    
    return similar_df

title = 'workstation verycapableforallcategories'

Recomm_System(title)

7


,CpuName,cosine_similarity
8,Core i9 12900K,1.0
9,Core i9 12900KF,1.0
10,Core i9 12900KS,1.0
13,Ryzen 9 3950X,1.0
15,Core i7 12700K,1.0
16,Core i7 12700KF,1.0
17,Ryzen 9 5900X,1.0
18,Ryzen 9 5900,1.0
21,Ryzen 9 3900X,1.0


In [27]:
#making the function so it does not have to work with the same exact string entering 
from fuzzywuzzy import process
def Recomm_System_v2(title):
    a=CPUclean.copy()
    matches = process.extractOne(title, a['Features'])
    if matches[1]<30:
       return 'No Close match found'
    index = a[a['Features'] == matches[0]].index[0]
    top_n_index = list(Cosine_sim_df[index].nlargest(10).index)
    try:
        top_n_index.remove(index)
    except:
        pass
    similar_df = a.iloc[top_n_index][['CpuName']]
    similar_df['cosine_similarity'] = Cosine_sim_df[index].iloc[top_n_index]
    return similar_df 
    
# You can see the difference in the input it does not have to be the same exact as in the dataset    
title = 'workstation very capable for all categories'
Recomm_System_v2(title)

c:\Users\Abdullah\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,CpuName,cosine_similarity
8,Core i9 12900K,1.0
9,Core i9 12900KF,1.0
10,Core i9 12900KS,1.0
13,Ryzen 9 3950X,1.0
15,Core i7 12700K,1.0
16,Core i7 12700KF,1.0
17,Ryzen 9 5900X,1.0
18,Ryzen 9 5900,1.0
21,Ryzen 9 3900X,1.0


In [84]:
# adding filtering with the price so it give the recommendation based on user's budget
# so the CPU price will be in range and capable for the user to buy 
def Recomm_System_v3(title,min_price=None, max_price=None):
    a=CPUclean.copy()
    matches = process.extractOne(title, a['Features'])
    if matches[1]<30:
       return 'No Close match found'
    index = a[a['Features'] == matches[0]].index[0]
    top_n_index = list(Cosine_sim_df[index].nlargest(20).index)
    try:
        top_n_index.remove(index)
    except:
        pass
    if min_price is not None or max_price is not None:
        filtered_indices = []
        for idx in top_n_index:
            price = float(a.loc[idx, 'Price'])  
            if (min_price is None or price >= min_price) and (max_price is None or price <= max_price):
                filtered_indices.append(idx)
        top_n_index = filtered_indices
    similar_df = a.iloc[top_n_index][['CpuName','Price']]
    similar_df['cosine_similarity'] = Cosine_sim_df[index].iloc[top_n_index]
    return similar_df 
    
title = 'workstation verycapableforallcategories'
Recomm_System_v3(title,400,900)

,CpuName,Price,cosine_similarity
13,Ryzen 9 3950X,459.99,1.0
15,Core i7 12700K,409,1.0
30,Core i7 12700,409,1.0
33,Core i9 10920X,757.9,1.0
43,Core i9 10900X,605,1.0
29,Core i9 11900KF,415.99,0.5


In [28]:
#in this last version it sorts the CPUs based on thier benchmark score in two crtieria (single thread score, multi threads score)
#from well-known test called Cinabench r23 
def Recomm_System_v4(title,min_price=None, max_price=None):
    a=CPUclean.copy()
    matches = process.extractOne(title, a['Features'])
    if matches[1]<30:
       return 'No Close match found'
    index = a[a['Features'] == matches[0]].index[0]
    top_n_index = list(Cosine_sim_df[index].nlargest(20).index)
    try:
        top_n_index.remove(index)
    except:
        pass
    if min_price is not None or max_price is not None:
        filtered_indices = []
        for idx in top_n_index:
            price = float(a.loc[idx, 'Price'])  
            if  (price <= max_price):
                filtered_indices.append(idx)
        top_n_index = filtered_indices
    similar_df = a.iloc[top_n_index][['CpuName', 'Price', 'SingleScore', 'MultiScore']]
    similar_df['cosine_similarity'] = Cosine_sim_df[index].iloc[top_n_index]
    similar_df = similar_df.sort_values(by=['SingleScore', 'MultiScore'], ascending=False)
    return similar_df
    
title = 'workstation very capable for all categories'
Recomm_System_v4(title,400,400).iloc[0][["CpuName","Price"]].tolist()

['Core i9 12900KS', '399.99']

In [20]:
HDD=pd.read_csv("HDD.csv")
def Get_HDD(budget):
    filtered_HDD = HDD[HDD["Price"]<=int(budget)]
    sorted_HDD = filtered_HDD.sort_values(by="Price",ascending=False)
    if sorted_HDD.empty:
        return "no matches for HDD"
    return sorted_HDD.iloc[0][["Name","Price","Size"]].tolist()

Get_HDD(-1)

'no matches for HDD'

In [62]:
case = pd.read_csv("case.csv")
#case = case.drop((case["type"] == "ATX Mid Tower") | (case["type"] == 'MicroATX Mini Tower') | (case["type"] == 'Mini ITX Desktop' | (case["type"] == 'ATX Mini Tower')) | (case["type"]=='HTPC')|(case["type"]=='MicroATX Slim')|(case["type"]=='ATX Desktop')|(case["type"]=='MicroATX Desktop')|(case["type"]=='Mini ITX Test Bench')|(case["type"]=='ATX Test Bench'))
case = case.drop(case[(case["type"] == "ATX Mid Tower") | (case["type"] == 'MicroATX Mini Tower') | (case["type"] == 'Mini ITX Desktop') | (case["type"] == 'ATX Mini Tower') | (case["type"]=='HTPC')|(case["type"]=='MicroATX Slim')|(case["type"]=='ATX Desktop')|(case["type"]=='MicroATX Desktop')|(case["type"]=='Mini ITX Test Bench')|(case["type"]=='ATX Test Bench')].index)

case = case.drop_duplicates()
case = case.dropna(subset=["price"])
case["type"].unique()

array(['MicroATX Mid Tower', 'ATX Full Tower', 'Mini ITX Tower'],
      dtype=object)

In [52]:
psu = pd.read_csv("power-supply.csv")
psu=psu.drop_duplicates()
psu = psu.dropna(subset=["price"])
psu['type'].unique()
psu = psu.drop(psu[(psu["type"]=='SFX') | (psu["type"]=='Flex ATX') | (psu["type"]=='TFX')].index)
psu.to_csv('modified_psu.csv',index=False)

In [61]:
cooler = pd.read_csv("cpu-cooler.csv")
cooler = cooler.drop_duplicates()
cooler = cooler.dropna(subset=["price"])
cooler=cooler.dropna(subset=["rpm"])
cooler=cooler.dropna(subset=["color"])
cooler=cooler.dropna(subset=["size"])
cooler.to_csv("modified_cooler.csv",index=False)